In [ ]:
! pip install pulp -q

Todo: Distance for in building walk

In [ ]:
import pulp as lp
import pickle

In [ ]:
with open("/kaggle/input/cs506-final-data-processed/small_exported_data.pkl", "rb") as f:
    data = pickle.load(f)
    print(type(data))
    print(data)

In [ ]:
minutes_in_week = 7 * 24 * 60
interval_duration = 5
max_intervals_in_week = minutes_in_week // interval_duration

In [ ]:
def decode_interval(interval):
    total_minutes = interval * interval_duration
    day_of_week = total_minutes // (24 * 60)
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    day_name = days_of_week[day_of_week]
    remaining_minutes = total_minutes % (24 * 60)
    hour = remaining_minutes // 60
    minute = remaining_minutes % 60
    return f"{hour}:{minute} {day_name}"

In [ ]:
num_professors = 5

num_classrooms = 10

classroom_capacities = data["capacities_top10"]

professor_courses = {0: [(150, 168, 31), (96, 117, 90), (672, 693, 90), (122, 143, 31), (186, 204, 30), (168, 186, 29), (978, 993, 32)], 1: [(180, 202, 21), (156, 178, 31), (1272, 1294, 55)], 2: [(474, 495, 50), (1050, 1071, 50)], 3: [(187, 197, 10), (763, 773, 10)], 4: [(384, 417, 30)]}

# Maybe add personal perference later, so use wrapped array for now
walking_cost = [data["walking_cost_df_top10"].values.tolist()]
print(walking_cost)

In [ ]:
problem = lp.LpProblem("Classroom_Assignment_Optimization", lp.LpMinimize)

# Whether professor i using j at time k
x = lp.LpVariable.dicts("x", ((i, j, k) for i in professor_courses.keys() for j in range(num_classrooms) for k in range(len(professor_courses[i]))), cat="Binary")

# Whether professor i move from classroom j to classroom m at the k
y = lp.LpVariable.dicts("y", ((i, j, k, m) for i in professor_courses.keys() for j in range(num_classrooms) for m in range(num_classrooms) for k in range(len(professor_courses[i]) - 1)), cat="Binary")

# Total walking cost
problem += lp.lpSum(
    walking_cost[0][j][m] * y[i, j, k, m]
    for i in professor_courses.keys()
    for j in range(num_classrooms)
    for m in range(num_classrooms)
    for k in range(len(professor_courses[i]) - 1)
)

# y[i, j, k, m] = x[i, j, k] * x[i, m, k + 1]
for i in professor_courses.keys():
    for j in range(num_classrooms):
        for m in range(num_classrooms):
            for k in range(len(professor_courses[i]) - 1):
                problem += y[i, j, k, m] <= x[i, j, k]
                problem += y[i, j, k, m] <= x[i, m, k + 1]
                problem += y[i, j, k, m] >= x[i, j, k] + x[i, m, k + 1] - 1

# One classroom per course at a time
for i in professor_courses.keys():
    for k, (start, end, _) in enumerate(professor_courses[i]):
        problem += lp.lpSum(x[i, j, k] for j in range(num_classrooms)) == 1

# One course per classroom at a time
for j in range(num_classrooms):
    for minute in range(max_intervals_in_week):
        problem += lp.lpSum(
            x[i, j, k]
            for i in professor_courses.keys()
            for k, (start, end, _) in enumerate(professor_courses[i])
            if start <= minute < end
        ) <= 1

# The classroom capacity must be equal to or larger than the demand
for i in professor_courses.keys():
    for k, (start, end, demand) in enumerate(professor_courses[i]):
        for j in range(num_classrooms):
            problem += x[i, j, k] * demand <= classroom_capacities[j]


In [ ]:
problem.solve()


In [ ]:
if lp.LpStatus[problem.status] == "Optimal":
    print("WE HAVE THE SOLUTION :D")
    for i in professor_courses.keys():
        for k in range(len(professor_courses[i])):
            for j in range(num_classrooms):
                if lp.value(x[i, j, k]) == 1:
                    start, end, demand = professor_courses[i][k]
                    print(f"Professor {i} is assigned to classroom {j} for course {k} from {decode_interval(start)} to {decode_interval(end)}")
else:
    print("THIS IS NOT GOOD, THERE IS NOT SOLUTION :|")